# Decision Tree on Smart Alert System

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score
)
from imblearn.over_sampling import SMOTE




## Process Data and Split 

In [ ]:
# Load the dataset
data_folder = Path.cwd() / "data"
csv_analysis = data_folder / "combinedAll.csv"
df = pd.read_csv(csv_analysis)

# Aggregate the data by 5-second intervals 
#Change interval to your desire
df['time_interval'] = (df['second'] - 1) // 5

aggregated_df = df.groupby('time_interval').agg({
    'distance': 'mean',
    'motion': 'mean',
    'lightIntensity': 'mean',
    'occupied': lambda x: x.mode().iloc[0],
    'occupied_note': lambda x: x.mode().iloc[0]
}).reset_index()

# Define features and target
X = aggregated_df[['distance', 'motion', 'lightIntensity', 'occupied']]
y = aggregated_df['occupied_note']

# Split into train and test sets (stratify ensures class balance)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


## Decision Tree

In [ ]:
# Define parameter grid for tuning
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Initialize Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Use GridSearchCV to find best parameters based on recall
grid_search = GridSearchCV(
    clf, param_grid, cv=5, scoring='precision', n_jobs=-1, verbose=1
) 
grid_search.fit(X_train_smote, y_train_smote)

# Get best model
best_clf = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")

# Train best model
best_clf.fit(X_train_smote, y_train_smote)

# Make predictions
y_pred = best_clf.predict(X_test)
y_proba = best_clf.predict_proba(X_test)[:, 1]

# Print classification report
print("
Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("
Confusion Matrix:")
print(cm)

# Compute ROC AUC score
roc_auc = roc_auc_score(y_test, y_proba)
print(f"
ROC AUC Score: {roc_auc:.2f}")

plt.figure(figsize=(12, 6))

# ROC Curve
plt.subplot(1, 2, 1)
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_proba)
avg_precision = average_precision_score(y_test, y_proba)
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"Precision-Recall Curve (AP = {avg_precision:.2f})", color='orange')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()

plt.tight_layout()
plt.show()

# Plot the decision tree
plt.figure(figsize=(20, 10))
plot_tree(
    best_clf,
    feature_names=X_train_smote.columns if hasattr(X_train_smote, 'columns') else None,
    class_names=['Not Occupied', 'Occupied'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Best Decision Tree")
plt.show()

## Random Forrest

In [ ]:
# Define parameter grid for tuning 
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [3, 5, 10, None],    # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],    # Minimum samples required at a leaf node
    'criterion': ['gini', 'entropy']  # Splitting criterion
}

# Initialize Random Forest Classifier
clf_rf = RandomForestClassifier(random_state=42)

# Use GridSearchCV to find best parameters based on precision
grid_search_rf = GridSearchCV(
    clf_rf, param_grid_rf, cv=5, scoring='precision', n_jobs=-1, verbose=1
)
grid_search_rf.fit(X_train_smote, y_train_smote)

# Get best model
best_clf_rf = grid_search_rf.best_estimator_
print(f"Best parameters: {grid_search_rf.best_params_}")

# Train best model
best_clf_rf.fit(X_train_smote, y_train_smote)

# Make predictions
y_pred_rf = best_clf_rf.predict(X_test)
y_proba_rf = best_clf_rf.predict_proba(X_test)[:, 1]

# Print classification report
print("
Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Print confusion matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
print("
Confusion Matrix:")
print(cm_rf)

# Compute ROC AUC score
roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
print(f"
ROC AUC Score: {roc_auc_rf:.2f}")

# Plot ROC Curve and Precision-Recall Curve
plt.figure(figsize=(12, 6))

# ROC Curve
plt.subplot(1, 2, 1)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)
plt.plot(fpr_rf, tpr_rf, label=f"ROC Curve (AUC = {roc_auc_rf:.2f})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()

# Precision-Recall Curve
precision_rf, recall_rf, _ = precision_recall_curve(y_test, y_proba_rf)
avg_precision_rf = average_precision_score(y_test, y_proba_rf)
plt.subplot(1, 2, 2)
plt.plot(recall_rf, precision_rf, label=f"Precision-Recall Curve (AP = {avg_precision_rf:.2f})", color='orange')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()

plt.tight_layout()
plt.show()

#Feature Impotance Plot

importances = best_clf_rf.feature_importances_
feature_names = X_train_smote.columns if hasattr(X_train_smote, 'columns') else [f'Feature {i}' for i in range(X_train_smote.shape[1])]
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances from Best Random Forest")
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45, ha='right')
plt.ylabel("Importance Score")
plt.tight_layout()
plt.show()